In [1]:
import pandas as pd
anime_df = pd.read_csv('MAL_data2.csv', index_col=0)
anime_df

,Fullmetal Alchemist: Brotherhood,Steins;Gate,Gintama°,Bleach: Sennen Kessen-hen,Shingeki no Kyojin Season 3 Part 2,Gintama',Hunter x Hunter (2011),Kaguya-sama wa Kokurasetai: Ultra Romantic,Gintama': Enchousen,Fruits Basket: The Final,...,DearS,Gegege no Kitarou (1985),Africa no Salaryman (TV),Android Ana Maico 2010,Battle Spirits: Sword Eyes,"Dakara Boku wa, H ga Dekinai.",Dekoboko Majo no Oyako Jijou,Tate no Yuusha no Nariagari Season 2,Teekyuu,Tenshi no Shippo Chu!
sinopsis,After a horrific alchemy experiment goes wrong...,Eccentric scientist Rintarou Okabe has a never...,"Gintoki, Shinpachi, and Kagura return as the f...",Substitute Soul Reaper Ichigo Kurosaki spends ...,Seeking to restore humanity's diminishing hope...,"After a one-year hiatus, Shinpachi Shimura ret...",Hunters devote themselves to accomplishing haz...,The elite members of Shuchiin Academy's studen...,"While Gintoki Sakata was away, the Yorozuya fo...","Hundreds of years ago, the Chinese zodiac spir...",...,"One year ago, a UFO containing 150 aliens cras...","As civilization advances, people are beginning...",Africa no Salaryman is a slapstick workplace c...,"In the year 2010, the Japanese radio industry ...","Many years ago, the death of the emperor spark...","On the surface, Ryousuke Kaga is the token per...","Alyssa, a witch who lives alone in the forest,...","With another Wave happening in a week, Naofumi...",Teekyuu is all about the wacky antics of four ...,"Somewhere, there was a young man. A kind heart..."
img_link,https://cdn.myanimelist.net/images/anime/1208/...,https://cdn.myanimelist.net/images/anime/1935/...,https://cdn.myanimelist.net/images/anime/3/720...,https://cdn.myanimelist.net/images/anime/1908/...,https://cdn.myanimelist.net/images/anime/1517/...,https://cdn.myanimelist.net/images/anime/4/503...,https://cdn.myanimelist.net/images/anime/1337/...,https://cdn.myanimelist.net/images/anime/1160/...,https://cdn.myanimelist.net/images/anime/1452/...,https://cdn.myanimelist.net/images/anime/1085/...,...,https://cdn.myanimelist.net/images/anime/1207/...,https://cdn.myanimelist.net/images/anime/11/27...,https://cdn.myanimelist.net/images/anime/1937/...,https://cdn.myanimelist.net/images/anime/8/303...,https://cdn.myanimelist.net/images/anime/11/49...,https://cdn.myanimelist.net/images/anime/4/751...,https://cdn.myanimelist.net/images/anime/1918/...,https://cdn.myanimelist.net/images/anime/1143/...,https://cdn.myanimelist.net/images/anime/1703/...,https://cdn.myanimelist.net/images/anime/12/44...
demographics,['Shounen'],NaN,['Shounen'],['Shounen'],['Shounen'],['Shounen'],['Shounen'],['Seinen'],['Shounen'],['Shoujo'],...,['Shounen'],NaN,NaN,['Seinen'],NaN,NaN,NaN,NaN,['Shounen'],NaN
score,9.10,9.07,9.06,9.05,9.05,9.04,9.04,9.04,9.03,8.99,...,6.56,6.56,6.55,6.55,6.55,6.55,6.55,6.55,6.55,6.55
genres,"['Action', 'Adventure', 'Drama', 'Fantasy']","['Drama', 'Sci-Fi', 'Suspense']","['Action', 'Comedy', 'Sci-Fi']","['Action', 'Adventure', 'Fantasy']","['Action', 'Drama', 'Suspense']","['Action', 'Comedy', 'Sci-Fi']","['Action', 'Adventure', 'Fantasy']","['Comedy', 'Romance']","['Action', 'Comedy', 'Sci-Fi']","['Drama', 'Romance', 'Supernatural']",...,"['Comedy', 'Romance', 'Sci-Fi', 'Ecchi']","['Comedy', 'Supernatural']",['Comedy'],"['Comedy', 'Drama', 'Sci-Fi']","['Adventure', 'Fantasy']","['Comedy', 'Romance', 'Supernatural', 'Ecchi']","['Comedy', 'Fantasy']","['Action', 'Adventure', 'Drama', 'Fantasy']","['Comedy', 'Sports']","['Fantasy', 'Romance']"
themes,Military,"Psychological, Time Travel","Gag Humor, Historical, Parody, Samurai",NaN,"Gore, Military, Survival","Gag Humor, Historical, Parody, Samurai",NaN,School,"Gag Humor, Historical, Parody, Samurai",NaN,...,Harem,NaN,"Anthropomorphic, Workplace","Adult Cast, Showbiz, Workplace",Strategy Game,NaN,Gag Humor,Isekai,School,NaN
episodes,64,24,51,13,10,51,148,13,13,13,...,12,108,12,24,29,12,12,13,12,11
status,Finished Airing,Finished Airing,Finished Airing,Finished Airing,Finished Airing,Finished

In [2]:
df = anime_df.T.copy()
df.sort_values(by='score', ascending = False, inplace=True)


In [3]:
import numpy as np
df['episodes'] = df['episodes'].replace('Unknown', np.nan)
float_columns = ['score','episodes', 'popularity', 'members', 'favorites']
df[float_columns] = df[float_columns].astype(float)

In [4]:
import json
df['genres'].fillna("['None']", inplace=True)
df['genres'] = df['genres'].apply(lambda x: json.loads(x.replace("'", "\"")))
df['genres'] = df['genres'].apply(lambda x: ', '.join(x))

In [5]:

# Concatenate the dummy columns with the original DataFrame
genre_columns = df['genres'].str.get_dummies(sep=', ')
df = pd.concat([df.drop(labels=['genres'],axis=1), genre_columns], axis=1)


themes_columns = df['themes'].str.get_dummies(sep=', ')
df = pd.concat([df.drop(labels=['themes'],axis=1), themes_columns], axis=1)


producers_columns = df['producers'].str.get_dummies(sep=', ')
df = pd.concat([df.drop(labels=['producers'],axis=1), producers_columns], axis=1)



In [6]:
from pycaret.regression import *
train = df.drop(labels=['sinopsis','img_link', 'demographics', 'status', 'aired','links' ], axis=1).sample(2500,
                                                                                                          random_state=1)
test = df.loc[~df.index.isin (train.index)]

In [7]:
# Train and Evaluate Model
genre_dummies = list(genre_columns.columns)
themes_dummies = list(themes_columns.columns)
producers_dummies = list(producers_columns.columns)


In [8]:

cat_features = genre_dummies+['studios', 'rating', 'licensors']
cat_features+=themes_dummies
cat_features+= producers_dummies
cat_features

['Action',
 'Adventure',
 'Avant Garde',
 'Award Winning',
 'Boys Love',
 'Comedy',
 'Drama',
 'Ecchi',
 'Fantasy',
 'Girls Love',
 'Gourmet',
 'Horror',
 'Mystery',
 'None',
 'Romance',
 'Sci-Fi',
 'Slice of Life',
 'Sports',
 'Supernatural',
 'Suspense',
 'studios',
 'rating',
 'licensors',
 'Adult Cast',
 'Anthropomorphic',
 'CGDCT',
 'Childcare',
 'Combat Sports',
 'Crossdressing',
 'Delinquents',
 'Detective',
 'Educational',
 'Gag Humor',
 'Gore',
 'Harem',
 'High Stakes Game',
 'Historical',
 'Idols (Female)',
 'Idols (Male)',
 'Isekai',
 'Iyashikei',
 'Love Polygon',
 'Magical Sex Shift',
 'Mahou Shoujo',
 'Martial Arts',
 'Mecha',
 'Medical',
 'Military',
 'Music',
 'Mythology',
 'Organized Crime',
 'Otaku Culture',
 'Parody',
 'Performing Arts',
 'Pets',
 'Psychological',
 'Racing',
 'Reincarnation',
 'Reverse Harem',
 'Romantic Subtext',
 'Samurai',
 'School',
 'Showbiz',
 'Space',
 'Strategy Game',
 'Super Power',
 'Survival',
 'Team Sports',
 'Time Travel',
 'Vampire',
 'V

In [9]:
train = train.loc[:,~train.columns.duplicated()].copy()

In [10]:
final_cat = pd.Series(cat_features).drop_duplicates().tolist()
experiment = setup(train, target='score', categorical_features=final_cat)


,Description,Value
0,Session id,7048
1,Target,score
2,Target type,Regression
3,Original data shape,"(2500, 1014)"
4,Transformed data shape,"(2500, 1019)"
5,Transformed train set shape,"(1750, 1019)"
6,Transformed test set shape,"(750, 1019)"
7,Ordinal features,840
8,Numeric features,4
9,Categorical features,1008


In [11]:
best_model = compare_models()


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.2569,0.1058,0.3247,0.6160,0.0386,0.0350,3.2050
dummy,Dummy Regressor,0.4262,0.2827,0.5311,-0.0073,0.0626,0.0576,2.6740


In [12]:
evaluate_model(best_model)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [21]:
test = test.loc[:,~test.columns.duplicated()].copy()
predict = predict_model(best_model, test)


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Light Gradient Boosting Machine,0.2343,0.0862,0.2936,0.6598,0.0353,0.0322


In [28]:
predict['error'] = abs(predict['score'] - predict['prediction_label'] )
predict['error_%'] =  abs(predict['prediction_label']/predict['score']-1)
predict[['error','error_%']].describe()

,error,error_%
count,500.000000,500.000000
mean,0.234314,0.032188
std,0.177063,0.024257
min,0.000250,0.000033
25%,0.101095,0.013488
50%,0.195728,0.026969
75%,0.331922,0.046666
max,0.993183,0.144149
